In [1]:
from huggingface_hub import login
login()

In [3]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import peft
from accelerate import Accelerator

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
def generate_text(model, tokenizer, prompt, max_length=50, num_return_sequences=1, device="cuda"):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.9,
        )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


In [4]:
base_model_path = "meta-llama/Llama-2-7b-chat-hf"
adapter_path = "adapter"
accelerator = Accelerator()
device = accelerator.device

In [7]:
!pip install accelerate

In [5]:
model_base = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype=torch.bfloat16)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [8]:
model = peft.PeftModel.from_pretrained(model_base, adapter_path).to(device)

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

ValueError: Can't find 'adapter_config.json' at 'adapter'

In [7]:
print("Model loaded successfully. You can start interacting with it.")

Model loaded successfully. You can start interacting with it.


In [ ]:
while True:
    prompt = input("Enter your prompt (type 'exit' to quit): ")
    if prompt.lower() == 'exit':
        break

    generated_texts = generate_text(model, tokenizer, prompt, max_length=50, num_return_sequences=1, device=device)
    for idx, text in enumerate(generated_texts):
        print(f"Generated Text {idx+1}: {text}")